In [1]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import glob
import cv2
import locale
import os

### Load in nnhe array Data

In [2]:
meta = []

with open('data-he.csv') as f:
    for line in f.readlines():
        meta.append(eval(line))

meta = np.array(meta)
np.save('nnhe-meta-simple-3', meta)

NameError: name 'np' is not defined

### Load in player hand data

In [5]:
player_hands = []

with open('data-ph.csv') as f:
    for line in f.readlines():
        player_hands.append(eval(line))

player_hands = np.array([arr.flatten() for arr in np.array(player_hands)])
np.save('nnhe-player-hands-simple-3', player_hands)

### From np file

In [2]:
meta = np.load('nnhe-meta-simple-3.npy')
player_hands = np.load('nnhe-player-hands-simple-3.npy')

### split train and test data

In [3]:
i = int(len(meta) / 2)
j = int(len(meta) * 3 / 4)

trainX = meta[:i]
testX = meta[i:j]
endX = meta[j:]

trainY = player_hands[:i]
testY = player_hands[i:j]
endY = player_hands[j:]

### Create model

In [4]:
model = Sequential()
model.add(Dense(100, input_dim=trainX.shape[1], activation="relu"))
model.add(Dense(85, activation="sigmoid"))
model.add(Dense(60, activation="sigmoid"))
model.add(Dense(52, activation="sigmoid"))

opt = Adam(
       learning_rate = 0.001,     # 0.001
       beta_1=0.9,                # 0.9
       beta_2=0.999,              # 0.999
       epsilon=0.0001,            # 1e-7
       amsgrad=False,             # False
       name="Adam"
      )

model.compile(loss='mean_squared_error', optimizer=opt, metrics = ['accuracy', 'mean_squared_error', 'categorical_crossentropy'])

### Fit

In [5]:
model.fit(
        trainX, trainY,
        validation_data=(testX, testY),
        epochs=200, batch_size=50
)

Epoch 1/200
25492/25492 [==============================] - 25s 984us/step - loss: 0.1374 - accuracy: 0.0295 - mean_squared_error: 0.1374 - categorical_crossentropy: 37.0836 - val_loss: 0.1295 - val_accuracy: 0.0301 - val_mean_squared_error: 0.1295 - val_categorical_crossentropy: 36.0970
Epoch 2/200
25492/25492 [==============================] - 25s 997us/step - loss: 0.1273 - accuracy: 0.0353 - mean_squared_error: 0.1273 - categorical_crossentropy: 35.7920 - val_loss: 0.1261 - val_accuracy: 0.0336 - val_mean_squared_error: 0.1261 - val_categorical_crossentropy: 35.6052
Epoch 3/200
25492/25492 [==============================] - 25s 998us/step - loss: 0.1247 - accuracy: 0.0363 - mean_squared_error: 0.1247 - categorical_crossentropy: 35.3956 - val_loss: 0.1245 - val_accuracy: 0.0363 - val_mean_squared_error: 0.1245 - val_categorical_crossentropy: 35.3201
Epoch 4/200
25492/25492 [==============================] - 25s 997us/step - loss: 0.1235 - accuracy: 0.0375 - mean_squared_error: 0.1235

In [6]:
model.save('nnhe-simple-3.h5')

# End - Testing

In [7]:
def largest(arr, n):
    new_arr = [*arr]
    for i in range(n):
        new_arr[new_arr.index(max(new_arr))] = -1
    return [1 if new_arr[i] == -1 else 0 for i in range(52)]

In [8]:
t = 6

X = endX
Y = endY
P = model.predict(np.array([X[t]]))[0]


np.set_printoptions(suppress=True)

print("test case:", t)
# print("stage:", sum(list(X[t][i]).count(-1) + list(X[t][i]).count(1) for i in range(4)))
print()
print("meta:")
print(X[t])
print()
print("Prediction:")
print(P)
print()
print("  pred:"," ".join('{:.2f}'.format(round(e,2)) for e in P))
print("actual:", " ".join(f"{e}   "[:4] for e in Y[t]))
print("\n", "-" * 150, "\n")
print("rounded:", *[int(round(p)) for p in P])
print(" actual:", *[e for e in Y[t]])
print("largest:", *largest(P, 15))
print()
print("mean squared diff:", sum((P - Y[t]) ** 2) / 52)

test case: 6

meta:
[ 0.29662436  0.27596366  0.          0.          0.29303631  0.25256771
  0.          0.28690141  0.32443202  0.55803436  0.40167862  0.21607871
  0.          0.26805452  0.23720407  0.26896271  0.25293341  0.
  0.          0.28942245  0.26915795  0.26233712  0.38060537  0.
  0.          0.          0.28537956  0.          0.26246625  0.26583919
  0.27335149  0.26849404  0.31981823  0.21549885  0.2038746   0.
  0.          0.          0.040843    0.24832146  0.22972237  0.
  0.27820095  0.24415258  0.27081439  0.28915071  0.26639208  0.21631734
  0.24274331  0.11261521  0.11096377  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          1.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          